# 第8篇：重塑和透视表

**本节内容：**
- 透视 df.pivot / pd.pivot_table
- 堆叠 stacking / unstacking
- 数据融合 (melt)
- 交叉表 crosstab()
- 分解 pd.factorize(x, sort=True)
- 虚拟对象 pd.get_dummies(df)
- 爆炸 df.explode('values')
- 窗口计算 rolling() expanding()

## 第一部分：数据透视 Pivot Table
数据透视是最常用的数据汇总工具，Excel 中经常会做数据透视，它可以根据一个或者多个指定的维度来聚合数据。Pandas 也提供了数据透视函数来实现这些功能。

### 1. pivot
![](https://zhangyafei-1258643511.cos.ap-nanjing.myqcloud.com/Python/blog/reshaping_pivot.png)

> pivot(index=None, columns=None, values=None)

这里有三个参数，作用分别是：
- index：新 df 的索引列，用于分组，如果为None，则使用现有索引
- columns：新 df 的列，如果透视后有重复值会报错
- values：用于填充 df 的列。 如果未指定，将使用所有剩余的列，并且结果将具有按层次结构索引的列

导入相关包

In [1]:
import pandas as pd
import numpy as np

数据准备

In [2]:
df = pd.DataFrame({'foo': ['one', 'one', 'one', 'two', 'two',
                           'two'],
                   'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'baz': [1, 2, 3, 4, 5, 6],
                   'zoo': ['x', 'y', 'z', 'q', 'w', 't']})
df

,foo,bar,baz,zoo
0,one,A,1,x
1,one,B,2,y
2,one,C,3,z
3,two,A,4,q
4,two,B,5,w
5,two,C,6,t


透视

In [3]:
df.pivot(index='foo', columns='bar', values='baz')

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


多层索引，取其中一列

In [4]:
df.pivot(index='foo', columns='bar')['baz']

bar,A,B,C
foo,,,
one,1,2,3
two,4,5,6


指定值

In [5]:
df.pivot(index='foo', columns='bar', values=['baz', 'zoo'])

baz       zoo      
bar   A  B  C   A  B  C
foo                    
one   1  2  3   x  y  z
two   4  5  6   q  w  t

再来看一个例子

In [6]:
index = pd.Index(data=["Tom", "Bob", "Mary", "James", "Andy", "Alice", 'Kobe','Yafei'], name="name")
data = {
    "age":  [18, 30, 35, 19, 22, 30, 37, 25],
    "city": ["北京", "上海", "广州", "克利夫兰", "郑州", "晋城", "洛杉矶", "晋城"],
    "sex": ["male", "male", "female", "male", "female", "female", "male", "male"],
    "income": [3000, 8000, 8000, 4000, 6000, 7000, 10000, 70000]
}
user_info = pd.DataFrame(data=data, index=index)
user_info

,age,city,sex,income
name,,,,
Tom,18,北京,male,3000
Bob,30,上海,male,8000
Mary,35,广州,female,8000
James,19,克利夫兰,male,4000
Andy,22,郑州,female,6000
Alice,30,晋城,female,7000
Kobe,37,洛杉矶,male,10000
Yafei,25,晋城,male,70000


一般状态下，数据在DataFrame会以压缩（stacked）状态存放，例如上面的sex，两个类别被叠在一列中，pivot函数可将某一列作为新的cols：

In [7]:
user_info.pivot(index='age', columns='sex', values='city')

sex,female,male
age,,
18,NaN,北京
19,NaN,克利夫兰
22,郑州,NaN
25,NaN,晋城
30,晋城,上海
35,广州,NaN
37,NaN,洛杉矶


pivot函数具有很强的局限性，它不允许values中出现重复的行列索引对（pair）。例如我把Tom的年龄修改为30，那么年龄为30，性别为male就会出现两个元素Tom和Bob,以下操作就会报错。

In [8]:
user_info.loc['Tom', 'age'] = 30
user_info

,age,city,sex,income
name,,,,
Tom,30,北京,male,3000
Bob,30,上海,male,8000
Mary,35,广州,female,8000
James,19,克利夫兰,male,4000
Andy,22,郑州,female,6000
Alice,30,晋城,female,7000
Kobe,37,洛杉矶,male,10000
Yafei,25,晋城,male,70000


In [9]:
# user_info.pivot(index='age', columns='sex', values='income')
# ValueError: Index contains duplicate entries, cannot reshape

### 2. pivot_table
df.pivot() 只能将数据进行整理，如果遇到重复值要进行聚合计算，就要用到pd.pivot_table()。它可以实现类似 Excel 那样的高级数据透视功能。
![](https://zhangyafei-1258643511.cos.ap-nanjing.myqcloud.com/Python/blog/reshaping_pivot_table.jpg)

> pivot_table(values=None,index=None,columns=None,aggfunc="mean",fill_value=None,margins=False,dropna=True,margins_name="All",observed=False,  

一些参数介绍：  
- data: 要透视的 DataFrame 对象  
- values: 要聚合的列或者多个列  
- index: 在数据透视表索引上进行分组的键  
- columns: 在数据透视表列上进行分组的键  
- aggfunc: 用于聚合的函数, 默认是 numpy.mean  

In [10]:
user_info.pivot_table(index='age', columns='sex', values='income')

sex,female,male
age,,
19,NaN,4000.0
22,6000.0,NaN
25,NaN,70000.0
30,7000.0,5500.0
35,8000.0,NaN
37,NaN,10000.0


In [11]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
                         "bar", "bar", "bar", "bar"],
                   "B": ["one", "one", "one", "two", "two",
                         "one", "one", "two", "two"],
                   "C": ["small", "large", "large", "small",
                         "small", "large", "small", "small",
                         "large"],
                   "D": [1, 2, 2, 3, 3, 4, 5, 6, 7],
                   "E": [2, 4, 5, 5, 6, 6, 8, 9, 9]})
df

,A,B,C,D,E
0,foo,one,small,1,2
1,foo,one,large,2,4
2,foo,one,large,2,5
3,foo,two,small,3,5
4,foo,two,small,3,6
5,bar,one,large,4,6
6,bar,one,small,5,8
7,bar,two,small,6,9
8,bar,two,large,7,9


将 D 列值加和，索引为 AB，列为 C 不去重值：

In [12]:
table = pd.pivot_table(df, values='D', index=['A', 'B'],
                    columns=['C'], aggfunc=np.sum)
table

C        large  small
A   B                
bar one    4.0    5.0
    two    7.0    6.0
foo one    4.0    1.0
    two    NaN    6.0

空值的传入

In [13]:
table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': np.mean})
table

D         E
A   C                        
bar large  5.500000  7.500000
    small  5.500000  8.500000
foo large  2.000000  4.500000
    small  2.333333  4.333333

不同值使用不同的聚合计算方式：

In [14]:
table = pd.pivot_table(df, values=['D', 'E'], index=['A', 'C'],
                    aggfunc={'D': np.mean,
                             'E': [min, max, np.mean]})
table

D    E               
               mean  max      mean  min
A   C                                  
bar large  5.500000  9.0  7.500000  6.0
    small  5.500000  9.0  8.500000  8.0
foo large  2.000000  5.0  4.500000  4.0
    small  2.333333  6.0  4.333333  2.0

汇总边际，给列的每层加一个 all 列进行汇总，计算方式与 aggfunc 相同。

In [15]:
pd.pivot_table(df, values='D', index=['A', 'B'],
               columns=['C'],  aggfunc=np.sum,
               margins=True)

C        large  small  All
A   B                     
bar one    4.0    5.0    9
    two    7.0    6.0   13
foo one    4.0    1.0    5
    two    NaN    6.0    6
All       15.0   18.0   33

index、columns和values均可以设置多个值

In [16]:
pd.pivot_table(df, values=['D', 'E'], index=['A', 'B'],
               columns=['C'],  aggfunc=np.sum,
               margins=True)

D               E          
C       large small All large small All
A   B                                  
bar one   4.0   5.0   9   6.0   8.0  14
    two   7.0   6.0  13   9.0   9.0  18
foo one   4.0   1.0   5   9.0   2.0  11
    two   NaN   6.0   6   NaN  11.0  11
All      15.0  18.0  33  24.0  30.0  54

## 第二部分：数据堆叠 Stack
如果原始数据集中的行列索引中均为层次索引，stack 过程表示将数据集的列旋转为行，同样 unstack 过程表示将数据的行旋转为列。

堆叠和取消堆叠
下面这堆叠的逻辑图示：
![](https://zhangyafei-1258643511.cos.ap-nanjing.myqcloud.com/Python/blog/reshaping_stack.png)

取消堆叠的示例：

![](https://zhangyafei-1258643511.cos.ap-nanjing.myqcloud.com/Python/blog/reshaping_unstack.png)

这些方法本质上是：

stack：“透视”某个级别的（可能是多层的）列标签，返回带有索引的 DataFrame，该索引带有一个新的最里面的行标签。
unstack：（堆栈的逆操作）将（可能是多层的）行索引的某个级别“透视”到列轴，从而生成具有新的最里面的列标签级别的重构的 DataFrame。
stack 过程将数据集的列转行，unstack 过程为行转列。

上例中，原始数据集索引有两层，堆叠过程就是将最列转到最内测的行上，unstack 是将最内层的行转移到最内层的列索引中。

In [17]:
df_single_level_cols = pd.DataFrame([[0, 1], [2, 3]],
                                    index=['cat', 'dog'],
                                    columns=['weight', 'height'])
df_single_level_cols

,weight,height
cat,0,1
dog,2,3


单层索引

In [18]:
df_single_level_cols.stack()

cat  weight    0
     height    1
dog  weight    2
     height    3
dtype: int64

多层索引

In [19]:
multicol1 = pd.MultiIndex.from_tuples([('weight', 'kg'),
                                       ('weight', 'pounds')])
df_multi_level_cols1 = pd.DataFrame([[1, 2], [2, 4]],
                                    index=['cat', 'dog'],
                                    columns=multicol1)

df_multi_level_cols1

weight       
        kg pounds
cat      1      2
dog      2      4

In [20]:
df_multi_level_cols1.stack()

weight
cat kg           1
    pounds       2
dog kg           2
    pounds       4

缺失值：

In [21]:
multicol2 = pd.MultiIndex.from_tuples([('weight', 'kg'),
                                       ('height', 'm')])
df_multi_level_cols2 = pd.DataFrame([[1.0, 2.0], [3.0, 4.0]],
                                    index=['cat', 'dog'],
                                    columns=multicol2)

df_multi_level_cols2

,weight,height
,kg,m
cat,1.0,2.0
dog,3.0,4.0


In [22]:
df_multi_level_cols2.stack()

height  weight
cat kg     NaN     1.0
    m      2.0     NaN
dog kg     NaN     3.0
    m      4.0     NaN

指定索引层级：

In [23]:
df_multi_level_cols2.stack(0)

kg    m
cat height  NaN  2.0
    weight  1.0  NaN
dog height  NaN  4.0
    weight  3.0  NaN

In [24]:
df_multi_level_cols2.stack([0, 1])

cat  height  m     2.0
     weight  kg    1.0
dog  height  m     4.0
     weight  kg    3.0
dtype: float64

删除缺失值：

In [25]:
df_multi_level_cols3 = pd.DataFrame([[None, 1.0], [2.0, 3.0]],
                                    index=['cat', 'dog'],
                                    columns=multicol2)

df_multi_level_cols3

,weight,height
,kg,m
cat,NaN,1.0
dog,2.0,3.0


In [26]:
df_multi_level_cols3.stack(dropna=False)

height  weight
cat kg     NaN     NaN
    m      1.0     NaN
dog kg     NaN     2.0
    m      3.0     NaN

In [27]:
df_multi_level_cols3.stack(dropna=True)

height  weight
cat m      1.0     NaN
dog kg     NaN     2.0
    m      3.0     NaN

**取消堆叠 unstack**

In [28]:
index = pd.MultiIndex.from_tuples([('one', 'a'), ('one', 'b'),
                                   ('two', 'a'), ('two', 'b')])
s = pd.Series(np.arange(1.0, 5.0), index=index)
s

one  a    1.0
     b    2.0
two  a    3.0
     b    4.0
dtype: float64

In [29]:
s.unstack(level=-1)

,a,b
one,1.0,2.0
two,3.0,4.0


In [30]:
s.unstack(level=0)

,one,two
a,1.0,3.0
b,2.0,4.0


In [31]:
df = s.unstack(level=0)
df.unstack()

one  a    1.0
     b    2.0
two  a    3.0
     b    4.0
dtype: float64

## 第三部分：交叉表 Crosstab
交叉表是用于统计分组频率的特殊透视表。简单来说，就是将两个或者多个列重中不重复的元素组成一个新的 DataFrame，新数据的行和列交叉的部分值为其组合在原数据中的数量。

语法结构如下：

> pd.crosstab(index, columns, values=None, rownames=None,
colnames=None, aggfunc=None, margins=False,
margins_name: str = 'All', dropna: bool = True,
normalize=False) → 'DataFrame'

参数说明：
- index：类数组，在行中按分组的值。
- columns：类数组的值，用于在列中进行分组。
- values：类数组的，可选的，要根据因素汇总的值数组。
- aggfunc：函数，可选，如果未传递任何值数组，则计算频率表。
- rownames：序列，默认为None，必须与传递的行数组数匹配。
- colnames：序列，默认值为None，如果传递，则必须与传递的列数组数匹配。
- margins：布尔值，默认为False，添加行/列边距（小计）
- normalize：布尔值，{'all'，'index'，'columns'}或{0,1}，默认为False。 通过将所有值除以值的总和进行归一化。

In [32]:
user_info

,age,city,sex,income
name,,,,
Tom,30,北京,male,3000
Bob,30,上海,male,8000
Mary,35,广州,female,8000
James,19,克利夫兰,male,4000
Andy,22,郑州,female,6000
Alice,30,晋城,female,7000
Kobe,37,洛杉矶,male,10000
Yafei,25,晋城,male,70000


交叉表是一种特殊的透视表，典型的用途如分组统计，如现在想要统计关于性别和年龄分组的频数：

In [33]:
pd.crosstab(index=user_info['sex'],columns=user_info['age'])

age,19,22,25,30,35,37
sex,,,,,,
female,0,1,0,1,1,0
male,1,0,1,2,0,1


values和aggfunc：分组对某些数据进行聚合操作，这两个参数必须成对出现

In [34]:
pd.crosstab(index=user_info['sex'],columns=user_info['age'], values=user_info['income'], aggfunc='mean')
#默认参数等于如下方法：
# pd.crosstab(index=user_info['sex'],columns=user_info['age'], values=1,aggfunc='count')

age,19,22,25,30,35,37
sex,,,,,,
female,NaN,6000.0,NaN,7000.0,8000.0,NaN
male,4000.0,NaN,70000.0,5500.0,NaN,10000.0


normalize参数，数据归一化，可选'all','index','columns'参数值

In [35]:
pd.crosstab(index=user_info['sex'],columns=user_info['age'], normalize='all')

age,19,22,25,30,35,37
sex,,,,,,
female,0.000,0.125,0.000,0.125,0.125,0.000
male,0.125,0.000,0.125,0.250,0.000,0.125


In [36]:
pd.crosstab(index=user_info['sex'],columns=user_info['age'], normalize='columns')

age,19,22,25,30,35,37
sex,,,,,,
female,0.0,1.0,0.0,0.333333,1.0,0.0
male,1.0,0.0,1.0,0.666667,0.0,1.0


In [37]:
pd.crosstab(index=user_info['sex'],columns=user_info['age'], normalize='index')

age,19,22,25,30,35,37
sex,,,,,,
female,0.0,0.333333,0.0,0.333333,0.333333,0.0
male,0.2,0.000000,0.2,0.400000,0.000000,0.2


边距汇总

In [38]:
pd.crosstab(index=user_info['sex'],columns=user_info['age'], normalize='all', margins=True)

age,19,22,25,30,35,37,All
sex,,,,,,,
female,0.000,0.125,0.000,0.125,0.125,0.000,0.375
male,0.125,0.000,0.125,0.250,0.000,0.125,0.625
All,0.125,0.125,0.125,0.375,0.125,0.125,1.000


## 第四部分：数据融合 Melt
df.melt() 是 df.pivot() 逆转操作函数。简单说就是将指定的列放到铺开放到行上名为variable(可指定)列，值在value(可指定)列。
  
具体语法结构如下：
> pd.melt(frame: pandas.core.frame.DataFrame,
        id_vars=None, value_vars=None,
        var_name='variable', value_name='value',
        col_level=None)

其中：
- id_vars: tuple，list或ndarray（可选），用作标识变量的列。
- value_vars: tuple，列表或ndarray，可选，要取消透视的列。 如果未指定，则使用未设置为id_vars的所有列。
- var_name: scalar，用于“变量”列的名称。 如果为None，则使用frame.columns.name或“variable”。
- value_name: scalar，默认为“ value”，用于“ value”列的名称。
- col_levelint或str，可选，如果列是MultiIndex，则使用此级别来融化。

In [39]:
user_info_reset_index = user_info.reset_index()
user_info_reset_index

,name,age,city,sex,income
0,Tom,30,北京,male,3000
1,Bob,30,上海,male,8000
2,Mary,35,广州,female,8000
3,James,19,克利夫兰,male,4000
4,Andy,22,郑州,female,6000
5,Alice,30,晋城,female,7000
6,Kobe,37,洛杉矶,male,10000
7,Yafei,25,晋城,male,70000


melt函数中的id_vars表示需要保留的列，value_vars表示需要stack的一组列

In [40]:
user_info_reset_index.melt(id_vars=['name'], value_vars=['sex'])

,name,variable,value
0,Tom,sex,male
1,Bob,sex,male
2,Mary,sex,female
3,James,sex,male
4,Andy,sex,female
5,Alice,sex,female
6,Kobe,sex,male
7,Yafei,sex,male


In [41]:
user_info_reset_index.melt(id_vars=['name', 'city'], value_vars=['sex', 'age'])

,name,city,variable,value
0,Tom,北京,sex,male
1,Bob,上海,sex,male
2,Mary,广州,sex,female
3,James,克利夫兰,sex,male
4,Andy,郑州,sex,female
5,Alice,晋城,sex,female
6,Kobe,洛杉矶,sex,male
7,Yafei,晋城,sex,male
8,Tom,北京,age,30
9,Bob,上海,age,30


## 第五部分：计算指示器/虚拟变量
Dummy Variables 即虚拟变量，又称虚设变量、名义变量或哑变量，用以反映质的属性的一个人工变量，是量化了的自变量，通常取值为0或1。经常用在 one-hot 特征提取。

语法结构如下：
> pd.get_dummies(data, prefix=None, 
               prefix_sep='_', dummy_na=False,
               columns=None, sparse=False,
               drop_first=False, dtype=None)

其中：
- prefix：新列的前缀
- prefix_sep：新列前缀的连接符

**逻辑说明**
简单说，pd.get_dummies() 是将一个或者多个列的去重值做为新表的列，每个列的值由0和1组成，在原来此位为此列名的值为1，不是的为0，这样就形成了一个由 0 和1 组成的特征矩阵。

In [42]:
pd.get_dummies(user_info['sex'])

,female,male
name,,
Tom,0,1
Bob,0,1
Mary,1,0
James,0,1
Andy,1,0
Alice,1,0
Kobe,0,1
Yafei,0,1


In [43]:
pd.get_dummies(user_info['sex'], prefix='sex')

,sex_female,sex_male
name,,
Tom,0,1
Bob,0,1
Mary,1,0
James,0,1
Andy,1,0
Alice,1,0
Kobe,0,1
Yafei,0,1


In [44]:
pd.get_dummies(user_info, columns=['sex'])

,age,city,income,sex_female,sex_male
name,,,,,
Tom,30,北京,3000,0,1
Bob,30,上海,8000,0,1
Mary,35,广州,8000,1,0
James,19,克利夫兰,4000,0,1
Andy,22,郑州,6000,1,0
Alice,30,晋城,7000,1,0
Kobe,37,洛杉矶,10000,0,1
Yafei,25,晋城,70000,0,1


## 第六部分：数据转置 df.T
理解数据转置
在数据处理分析过程中，为了充分利用行列的关系表达，我们需要对原数据的行列进行互换。转置的过程其实是沿着左上与右下形成对角线进行翻转。
![](https://zhangyafei-1258643511.cos.ap-nanjing.myqcloud.com/Python/blog/pandas-transpose.png)

**转置 df.T**  
df.T 属性是 df.transpose() 方法的别名、简写方法，今后我们只要记住 .T 就好啦。

In [45]:
user_info

,age,city,sex,income
name,,,,
Tom,30,北京,male,3000
Bob,30,上海,male,8000
Mary,35,广州,female,8000
James,19,克利夫兰,male,4000
Andy,22,郑州,female,6000
Alice,30,晋城,female,7000
Kobe,37,洛杉矶,male,10000
Yafei,25,晋城,male,70000


In [46]:
user_info_transposed = user_info.T  # user_info.transpose()
user_info_transposed

name,Tom,Bob,Mary,James,Andy,Alice,Kobe,Yafei
age,30,30,35,19,22,30,37,25
city,北京,上海,广州,克利夫兰,郑州,晋城,洛杉矶,晋城
sex,male,male,female,male,female,female,male,male
income,3000,8000,8000,4000,6000,7000,10000,70000


**轴交换 swapaxes**  
Pandas 提供了一个 DataFrame.swapaxes(axis1, axis2, copy=True) 用来做轴（行列）交换。如果行列交换就相当于 df.T。

In [47]:
user_info.swapaxes("index", "columns") # 行列交换，相当于 df.T
# user_info.swapaxes("column", "index") # 行列交换，相当于 df.T

name,Tom,Bob,Mary,James,Andy,Alice,Kobe,Yafei
age,30,30,35,19,22,30,37,25
city,北京,上海,广州,克利夫兰,郑州,晋城,洛杉矶,晋城
sex,male,male,female,male,female,female,male,male
income,3000,8000,8000,4000,6000,7000,10000,70000


## 第七部分：因子化（枚举化）值
因子化值是指将个一维的数据，由于在大量的重复值，可以解析成枚举值，这样我们就方便进行分辨。factorize 既可以用作顶层函数 pandas.factorize()，也可以用作Series.factorize() 和 Index.factorize() 方法。

### 基本方法
将一个方法进行因子化后将返回两个值，一个是因子化后的编码列表，一个是原数据的去重值列表：

In [48]:
col_code, col = pd.factorize(user_info['sex'])
col_code

array([0, 0, 1, 0, 1, 1, 0, 0], dtype=int64)

In [49]:
col

Index(['male', 'female'], dtype='object')

### 排序
使用 sort=True 参数后将对唯一性进行排序，编码列表将继续与原值保持对应关系，但从值的大小上将体现出顺序。

In [50]:
col_code, col = pd.factorize(user_info['sex'], sort=True)
print(col_code, col)

[1 1 0 1 0 0 1 1] Index(['female', 'male'], dtype='object')


### 缺失值
缺失值不会出现在唯一值列表中，在编码中将为 -1：

In [51]:
col_code, col = pd.factorize(['female', 'male', 'female', 'male', np.nan, 'male'], sort=True)
print(col_code, col)

[ 0  1  0  1 -1  1] ['female' 'male']


### 枚举类型
Categorical 枚举类型也可以使用此方法：

In [52]:
cat = pd.Categorical(['a', 'a', 'c'], categories=['a', 'b', 'c'])
col_code, col = pd.factorize(cat)
print(col_code, col)

[0 0 1] ['a', 'c']
Categories (3, object): ['a', 'b', 'c']


## 第八部分：爆炸序列
爆炸这个词非常形象，是指将类似列表的每个元素转换为一行，索引值是相同的，就这么简单，下边直接上代码。

### 基本用法
下边的两行数据中有类似列表（list-likes，包括 lists, tuples, sets, Series 和 np.ndarray）的值，我们将它们炸开后，它他乖乖回去排好了队，但是依然使用原来的索引：

In [53]:
s = pd.Series([[1, 2, 3], 'foo', [], [3, 4]])
s

0    [1, 2, 3]
1          foo
2           []
3       [3, 4]
dtype: object

In [54]:
s.explode()

0      1
0      2
0      3
1    foo
2    NaN
3      3
3      4
dtype: object

子集行的结果 dtype 将为 object。 标量将原封不动地返回，并且空列表状将导致该行的 np.nan。 此外，爆炸集合时，输出中行的顺序将不确定。

### DataFrame 的爆炸
我们看到，对指定列进行了炸裂：

In [55]:
df = pd.DataFrame({'A': [[1, 2, 3], 'foo', [], [3, 4]], 'B': 1})
df

,A,B
0,"[1, 2, 3]",1
1,foo,1
2,[],1
3,"[3, 4]",1


In [56]:
df.explode('A')

,A,B
0,1,1
0,2,1
0,3,1
1,foo,1
2,NaN,1
3,3,1
3,4,1


### 炸开非列表
有时候遇到不是列表的，但是具有列表的特质，我们也可以处理：

In [57]:
df = pd.DataFrame([{'var1': 'a,b,c', 'var2': 1},
                   {'var1': 'd,e,f', 'var2': 2}])
df

,var1,var2
0,"a,b,c",1
1,"d,e,f",2


看看 var1 列，我们发现可以处理成列表：

In [58]:
df.assign(var1=df.var1.str.split(',')).explode('var1')

,var1,var2
0,a,1
0,b,1
0,c,1
1,d,2
1,e,2
1,f,2


## 第九部分：转为 NumPy ndarray
众所周知，在特征处理和数据建模中，很多库使用的是 NumPy 的 ndarray 数据类型，Pandas 对数据处理后要应用到上述场景就需要将类型转为 NumPy 的 ndarray 。本文介绍如何将 Pandas 的 Series 和 Dataframe 转换为 NumPy 的 ndarray。
### 概述
pandas v0.24.0 引入了两种从 pandas 对象获取 NumPy 数组的新方法：

ds.to_numpy(), 它可以用在 Index, Series, 和 DataFrame 对象
s.array, 为 PandasArray，用在 Index 和 Series，它包装了 numpy.ndarray 接口
pandas 的 values 和 as_matrix() 不赞成使用。这两个函数旨在提高 API 的一致性，这是朝着正确方向迈出的重要一步。最后，.values 和 as_matrix() 在当前版本中不会被弃用，但预计这可能会在将来的某个时候发生，因此建议用户尽快迁移到较新的 API。

### Dataframe
df.values 和 df.to_numpy()返回的是一个 array 类型：

In [59]:
user_info.values # 不建议

array([[30, '北京', 'male', 3000],
       [30, '上海', 'male', 8000],
       [35, '广州', 'female', 8000],
       [19, '克利夫兰', 'male', 4000],
       [22, '郑州', 'female', 6000],
       [30, '晋城', 'female', 7000],
       [37, '洛杉矶', 'male', 10000],
       [25, '晋城', 'male', 70000]], dtype=object)

In [60]:
user_info.to_numpy()

array([[30, '北京', 'male', 3000],
       [30, '上海', 'male', 8000],
       [35, '广州', 'female', 8000],
       [19, '克利夫兰', 'male', 4000],
       [22, '郑州', 'female', 6000],
       [30, '晋城', 'female', 7000],
       [37, '洛杉矶', 'male', 10000],
       [25, '晋城', 'male', 70000]], dtype=object)

In [61]:
type(user_info.to_numpy())

numpy.ndarray

In [62]:
user_info.to_numpy().dtype

dtype('O')

指定列转换

In [63]:
user_info[['sex', 'age']].to_numpy()

array([['male', 30],
       ['male', 30],
       ['female', 35],
       ['male', 19],
       ['female', 22],
       ['female', 30],
       ['male', 37],
       ['male', 25]], dtype=object)

### Series
对 Series 使用 s.values 和 s.to_numpy()返回的是一个 array 类型：

In [64]:
user_info['age'].values  # 不建议

array([30, 30, 35, 19, 22, 30, 37, 25], dtype=int64)

In [65]:
user_info['age'].to_numpy()

array([30, 30, 35, 19, 22, 30, 37, 25], dtype=int64)

In [66]:
type(user_info['age'].to_numpy())

numpy.ndarray

In [67]:
user_info['age'].to_numpy().dtype

dtype('int64')

In [68]:
user_info['age'].array

<PandasArray>
[30, 30, 35, 19, 22, 30, 37, 25]
Length: 8, dtype: int64

In [69]:
type(user_info['age'].array)

pandas.core.arrays.numpy_.PandasArray

### df.to_records()
您可以使用 to_records() 方法，但是如果数据类型不是您想要的，则必须对它们进行一些处理。在下例子中，从字符串复制 df 之后，索引类型是 string（由 pandas 中的object dtype 表示）

In [70]:
user_info.to_records()

rec.array([('Tom', 30, '北京', 'male',  3000),
           ('Bob', 30, '上海', 'male',  8000),
           ('Mary', 35, '广州', 'female',  8000),
           ('James', 19, '克利夫兰', 'male',  4000),
           ('Andy', 22, '郑州', 'female',  6000),
           ('Alice', 30, '晋城', 'female',  7000),
           ('Kobe', 37, '洛杉矶', 'male', 10000),
           ('Yafei', 25, '晋城', 'male', 70000)],
          dtype=[('name', 'O'), ('age', '<i8'), ('city', 'O'), ('sex', 'O'), ('income', '<i8')])

In [71]:
type(user_info.to_records())

numpy.recarray

### NumPy 的方法
可以用 np.array 直接转换：

In [72]:
np.array(user_info)

array([[30, '北京', 'male', 3000],
       [30, '上海', 'male', 8000],
       [35, '广州', 'female', 8000],
       [19, '克利夫兰', 'male', 4000],
       [22, '郑州', 'female', 6000],
       [30, '晋城', 'female', 7000],
       [37, '洛杉矶', 'male', 10000],
       [25, '晋城', 'male', 70000]], dtype=object)

In [73]:
np.array(user_info['age'])

array([30, 30, 35, 19, 22, 30, 37, 25], dtype=int64)

In [74]:
np.array(user_info['age'].array)

array([30, 30, 35, 19, 22, 30, 37, 25], dtype=int64)

In [75]:
np.array(user_info.to_records())

array([('Tom', 30, '北京', 'male',  3000), ('Bob', 30, '上海', 'male',  8000),
       ('Mary', 35, '广州', 'female',  8000),
       ('James', 19, '克利夫兰', 'male',  4000),
       ('Andy', 22, '郑州', 'female',  6000),
       ('Alice', 30, '晋城', 'female',  7000),
       ('Kobe', 37, '洛杉矶', 'male', 10000),
       ('Yafei', 25, '晋城', 'male', 70000)],
      dtype=(numpy.record, [('name', 'O'), ('age', '<i8'), ('city', 'O'), ('sex', 'O'), ('income', '<i8')]))